In [3]:
import os
import sys
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
import logging

sys.path.append(os.pardir)
from database.database import Database
from config import SLACK_API_TOKEN, SLACK_CHANNEL, DATABASE_NAME
from utils.utilts import get_papers, fetch_interesting_points, fetch_summary
from main import post_to_slack

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.StreamHandler()],
)

logger = logging.getLogger(__name__)


client = WebClient(token=SLACK_API_TOKEN)

In [ ]:
db = Database(DATABASE_NAME)
db.init_database()

try:
    paper = get_papers(db)
    summary = fetch_summary(paper)
    interesting_points = fetch_interesting_points(paper)
    main_text = f"""
    *タイトル:* {paper.title}
    *著者名:* {paper.authors}
    *リンク:* {paper.url}
    """

    reply_text = f"""
    *概要:*\n{summary}
    *提出日:* {paper.submitted}
    *面白いポイント:*\n{interesting_points}
    ChatPDFで読む: https://www.chatpdf.com/
    """
    post_to_slack(main_text, reply_text)
    logger.info(f"Posted a paper: {paper.title}")
except Exception as e:
    logger.error(f"Error: {e}")